## XML parsing with ElementTree (+ XPath)

In [1]:
from xml.etree.ElementTree import ElementTree

In [2]:
with open("0104-6632-bjce-33-01-0001.xml") as f:
    doc_etree = ElementTree(file=f)

In [3]:
doc_etree.findall("//aff")

/usr/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: This search is broken in 1.3 and earlier, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/aff'
  """Entry point for launching an IPython kernel.


[<Element 'aff' at 0x7fa9f02ec098>]

In [4]:
print(doc_etree.findall("//aff")[0])

<Element 'aff' at 0x7fa9f02ec098>


/usr/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: This search is broken in 1.3 and earlier, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/aff'
  """Entry point for launching an IPython kernel.


## XML parsing with libxml2 (+ XPath)

In [5]:
import libxml2

In [6]:
doc = libxml2.parseFile("0104-6632-bjce-33-01-0001.xml")

In [7]:
doc_affs = doc.xpathEval("//aff")
doc_affs

[<xmlNode (aff) object at 0x7fa9f017a710>]

In [8]:
doc_aff = doc_affs[0]
print(doc_aff)

<aff id="aff1">
					<institution content-type="orgname">University of Chemical Technology and Metallurgy</institution>
					<addr-line>
						<named-content content-type="city">Sofia</named-content>
					</addr-line>
					<country country="BG">Bulgaria</country>
					<email>rteodosieva@abv.bg</email>
					<institution content-type="original">University of Chemical Technology and Metallurgy, 8 Kliment Ohridski Blvd. 1756, Sofia, Bulgaria. Phone: + 359 28163247. E-mail: rteodosieva@abv.bg</institution>
				</aff>


## XML parsing with lxml's ElementTree (+ XPath)

In [9]:
from lxml import etree

In [10]:
with open("0104-6632-bjce-33-01-0001.xml") as f:
    data = etree.parse(f)

In [11]:
data.xpath("//aff/country/text()[normalize-space()]")

['Bulgaria']

In [12]:
country_xpath = etree.XPath("//aff/country/text()[normalize-space()]")
country_xpath

//aff/country/text()[normalize-space()]

In [13]:
country_xpath(data)

['Bulgaria']

## XSLT with lxml

In [14]:
transform = etree.XSLT(etree.XML("""
<xsl:stylesheet version="1.0" xmlns:xsl="http://www.w3.org/1999/XSL/Transform">
    <xsl:output method="text" encoding="utf-8" />
    <xsl:template match="//a">
        <xsl:text>{"test": </xsl:text>
        <xsl:value-of select="b/text()" />
        <xsl:text>}</xsl:text>
    </xsl:template>
</xsl:stylesheet>
"""))

In [15]:
print(transform(etree.XML("<c><a><b>Tééeext</b></a></c>")))

{"test": Tééeext}


## XML to JSON with XSLT

XSLT from https://www.bjelic.net/2012/08/01/coding/convert-xml-to-json-using-xslt/

In [16]:
xml2json_xslt = etree.XSLT(etree.XML("""
<xsl:stylesheet version="1.0" xmlns:xsl="http://www.w3.org/1999/XSL/Transform">
    <xsl:output method="text" encoding="utf-8"/>
 
    <xsl:template match="/*[node()]">
        <xsl:text>{</xsl:text>
        <xsl:apply-templates select="." mode="detect" />
        <xsl:text>}</xsl:text>
    </xsl:template>
 
    <xsl:template match="*" mode="detect">
        <xsl:choose>
            <xsl:when test="name(preceding-sibling::*[1]) = name(current()) and name(following-sibling::*[1]) != name(current())">
                    <xsl:apply-templates select="." mode="obj-content" />
                <xsl:text>]</xsl:text>
                <xsl:if test="count(following-sibling::*[name() != name(current())]) &gt; 0">, </xsl:if>
            </xsl:when>
            <xsl:when test="name(preceding-sibling::*[1]) = name(current())">
                    <xsl:apply-templates select="." mode="obj-content" />
                    <xsl:if test="name(following-sibling::*) = name(current())">, </xsl:if>
            </xsl:when>
            <xsl:when test="following-sibling::*[1][name() = name(current())]">
                <xsl:text>"</xsl:text><xsl:value-of select="name()"/><xsl:text>" : [</xsl:text>
                    <xsl:apply-templates select="." mode="obj-content" /><xsl:text>, </xsl:text>
            </xsl:when>
            <xsl:when test="count(./child::*) > 0 or count(@*) > 0">
                <xsl:text>"</xsl:text><xsl:value-of select="name()"/>" : <xsl:apply-templates select="." mode="obj-content" />
                <xsl:if test="count(following-sibling::*) &gt; 0">, </xsl:if>
            </xsl:when>
            <xsl:when test="count(./child::*) = 0">
                <xsl:text>"</xsl:text><xsl:value-of select="name()"/>" : "<xsl:apply-templates select="."/><xsl:text>"</xsl:text>
                <xsl:if test="count(following-sibling::*) &gt; 0">, </xsl:if>
            </xsl:when>
        </xsl:choose>
    </xsl:template>
 
    <xsl:template match="*" mode="obj-content">
        <xsl:text>{</xsl:text>
            <xsl:apply-templates select="@*" mode="attr" />
            <xsl:if test="count(@*) &gt; 0 and (count(child::*) &gt; 0 or text())">, </xsl:if>
            <xsl:apply-templates select="./*" mode="detect" />
            <xsl:if test="count(child::*) = 0 and text() and not(@*)">
                <xsl:text>"</xsl:text><xsl:value-of select="name()"/>" : "<xsl:value-of select="text()"/><xsl:text>"</xsl:text>
            </xsl:if>
            <xsl:if test="count(child::*) = 0 and text() and @*">
                <xsl:text>"text" : "</xsl:text><xsl:value-of select="text()"/><xsl:text>"</xsl:text>
            </xsl:if>
        <xsl:text>}</xsl:text>
        <xsl:if test="position() &lt; last()">, </xsl:if>
    </xsl:template>
 
    <xsl:template match="@*" mode="attr">
        <xsl:text>"</xsl:text><xsl:value-of select="name()"/>" : "<xsl:value-of select="."/><xsl:text>"</xsl:text>
        <xsl:if test="position() &lt; last()">,</xsl:if>
    </xsl:template>
 
    <xsl:template match="node/@TEXT | text()" name="removeBreaks">
        <xsl:param name="pText" select="normalize-space(.)"/>
        <xsl:choose>
            <xsl:when test="not(contains($pText, '&#xA;'))"><xsl:copy-of select="$pText"/></xsl:when>
            <xsl:otherwise>
                <xsl:value-of select="concat(substring-before($pText, '&#xD;&#xA;'), ' ')"/>
                <xsl:call-template name="removeBreaks">
                    <xsl:with-param name="pText" select="substring-after($pText, '&#xD;&#xA;')"/>
                </xsl:call-template>
            </xsl:otherwise>
        </xsl:choose>
    </xsl:template>
 
</xsl:stylesheet>
"""))

In [17]:
print(xml2json_xslt(etree.XML("<c><a><b>Tééeext</b></a></c>")))

{"c" : {"a" : {"b" : "Tééeext"}}}


Does it with an article XML?

In [18]:
print(xml2json_xslt(data))

{"article" : {"article-type" : "research-article","dtd-version" : "1.0","specific-use" : "sps-1.4","xml:lang" : "en", "front" : {"journal-meta" : {"journal-id" : {"journal-id-type" : "publisher-id", "text" : "bjce"}, "journal-title-group" : {"journal-title" : "Brazilian Journal of Chemical Engineering", "abbrev-journal-title" : {"abbrev-type" : "publisher", "text" : "Braz. J. Chem. Eng."}}, "issn" : [{"pub-type" : "ppub", "text" : "0104-6632"}, {"pub-type" : "epub", "text" : "1678-4383"}], "publisher" : {"publisher-name" : "Brazilian Society of Chemical Engineering"}}, "article-meta" : {"article-id" : {"pub-id-type" : "doi", "text" : "10.1590/0104-6632.20160331s00003267"}, "article-categories" : {"subj-group" : {"subj-group-type" : "heading", "subject" : "BIOPROCESS ENGINEERING"}}, "title-group" : {"article-title" : "BIODECOMPOSITION OF JORDAN PHOSPHORITE BY PHOSPHATE-SOLUBILIZING FUNGI"}, "contrib-group" : {"contrib" : [{"contrib-type" : "author", "name" : {"surname" : "Teodosieva", "

In [19]:
import json

As it seems, it's broken...

In [20]:
json.loads(str(xml2json_xslt(data)))

JSONDecodeError: Expecting ',' delimiter: line 1 column 7815 (char 7814)

In [21]:
# with open("0104-6632-bjce-33-01-0001.json", "w") as f:
#     f.write(str(xml2json_xslt(data)))

## Left-joining aff and contrib with XSLT

In [22]:
for x in data.xpath("//contrib"):
    print(etree.tostring(x, encoding="utf-8").decode("utf-8"))

<contrib xmlns:mml="http://www.w3.org/1998/Math/MathML" xmlns:xlink="http://www.w3.org/1999/xlink" contrib-type="author">
					<name>
						<surname>Teodosieva</surname>
						<given-names>R.</given-names>
					</name>
					<xref ref-type="aff" rid="aff1"/>
					<xref ref-type="fn" rid="fn1">*</xref>
				</contrib>
				
<contrib xmlns:mml="http://www.w3.org/1998/Math/MathML" xmlns:xlink="http://www.w3.org/1999/xlink" contrib-type="author">
					<name>
						<surname>Bojinova</surname>
						<given-names>D.</given-names>
					</name>
					<xref ref-type="aff" rid="aff1"/>
				</contrib>
				


In [23]:
for x in data.xpath('//contrib/xref[@ref-type = "aff" and @rid = $affid]/..', affid="aff1"):
    print(etree.tostring(x, encoding="utf-8").decode("utf-8"))

<contrib xmlns:mml="http://www.w3.org/1998/Math/MathML" xmlns:xlink="http://www.w3.org/1999/xlink" contrib-type="author">
					<name>
						<surname>Teodosieva</surname>
						<given-names>R.</given-names>
					</name>
					<xref ref-type="aff" rid="aff1"/>
					<xref ref-type="fn" rid="fn1">*</xref>
				</contrib>
				
<contrib xmlns:mml="http://www.w3.org/1998/Math/MathML" xmlns:xlink="http://www.w3.org/1999/xlink" contrib-type="author">
					<name>
						<surname>Bojinova</surname>
						<given-names>D.</given-names>
					</name>
					<xref ref-type="aff" rid="aff1"/>
				</contrib>
				


In [24]:
aff_contrib_xslt = etree.XSLT(etree.XML("""
<xsl:stylesheet version="1.0" xmlns:xsl="http://www.w3.org/1999/XSL/Transform">
    <xsl:output method="xml" version="1.0" encoding="utf-8" indent="yes"/>
    <xsl:strip-space elements="*"/>

    <xsl:template match="article">
        <pairs>
        <xsl:for-each select="//aff">
            <xsl:variable name ="affid" select="@id" />
            <xsl:call-template name="left_join">
                <xsl:with-param name="aff" select="." />
                <xsl:with-param name="contrib" select="//contrib/xref[@ref-type = 'aff' and @rid = $affid]/.." />
            </xsl:call-template>
        </xsl:for-each>
        </pairs>
    </xsl:template>

    <xsl:template name="left_join">
        <xsl:param name="aff" />
        <xsl:param name="contrib" />

        <xsl:choose>
            <xsl:when test="$contrib">
                <xsl:for-each select="$contrib">
                    <xsl:call-template name="print_join">
                        <xsl:with-param name="aff" select="$aff" />
                        <xsl:with-param name="contrib" select="." />
                    </xsl:call-template>
                </xsl:for-each>
            </xsl:when>
            <xsl:otherwise>
                <xsl:call-template name="print_join">
                    <xsl:with-param name="aff" select="$aff" />
                </xsl:call-template>
            </xsl:otherwise>
        </xsl:choose>
    </xsl:template>

    <xsl:template match="node() | @*">
        <xsl:copy>
            <xsl:apply-templates select="node() | @*" />
        </xsl:copy>
    </xsl:template>

    <xsl:template name="print_join">
        <xsl:param name="aff" />
        <xsl:param name="contrib" />
        <xsl:element name="affcontrib">
            <xsl:for-each select="$aff">
                <xsl:apply-templates select="node()" />
            </xsl:for-each>
            <xsl:if test="$contrib">
                <xsl:apply-templates select="$contrib/name"/>
                <ctype><xsl:value-of select="$contrib/@contrib-type" /></ctype>
            </xsl:if>
        </xsl:element>
    </xsl:template>

</xsl:stylesheet>
"""))

In [25]:
print(aff_contrib_xslt(data))

<?xml version="1.0"?>
<pairs>
  <affcontrib>
    <institution content-type="orgname">University of Chemical Technology and Metallurgy</institution>
    <addr-line>
      <named-content content-type="city">Sofia</named-content>
    </addr-line>
    <country country="BG">Bulgaria</country>
    <email>rteodosieva@abv.bg</email>
    <institution content-type="original">University of Chemical Technology and Metallurgy, 8 Kliment Ohridski Blvd. 1756, Sofia, Bulgaria. Phone: + 359 28163247. E-mail: rteodosieva@abv.bg</institution>
    <name>
      <surname>Teodosieva</surname>
      <given-names>R.</given-names>
    </name>
    <ctype>author</ctype>
  </affcontrib>
  <affcontrib>
    <institution content-type="orgname">University of Chemical Technology and Metallurgy</institution>
    <addr-line>
      <named-content content-type="city">Sofia</named-content>
    </addr-line>
    <country country="BG">Bulgaria</country>
    <email>rteodosieva@abv.bg</email>
    <institution content-type="origi

## Where does the dataset have aff xrefs?

In [26]:
import pathlib, collections

In [27]:
fnames = collections.defaultdict(set)
fnames_total = 0
for xml_path in pathlib.Path("selecao_xml_br").glob("**/*.xml"):
    fnames_total += 1
    try:
        with open(xml_path) as xml_file:
            xml_etree = etree.parse(xml_file)
    except:
        fnames["<errored>"].add(xml_path)
    else:
        for el in xml_etree.xpath("//xref[@ref-type = 'aff']/.."):
            fnames[el.tag].add(xml_path)

In [28]:
quantities = {k: len(v) for k, v in fnames.items()}
quantities["<total>"] = fnames_total
quantities

{'<errored>': 425,
 '<total>': 23815,
 'aff': 2,
 'contrib': 21991,
 'name': 1,
 'p': 5}

In [29]:
quantities["<errored>"] / quantities["<total>"]

0.01784589544404787

In [30]:
from IPython.lib.pretty import pprint
pprint(fnames, max_seq_length=float("inf"))

defaultdict(set,
            {'<errored>': {PosixPath('selecao_xml_br/aa/v27n1/1809-4392-aa-27-1-0003.rep.xml'),
              PosixPath('selecao_xml_br/aa/v27n1/1809-4392-aa-27-1-0009.rep.xml'),
              PosixPath('selecao_xml_br/aa/v27n1/1809-4392-aa-27-1-0027.rep.xml'),
              PosixPath('selecao_xml_br/aa/v27n1/1809-4392-aa-27-1-0033.rep.xml'),
              PosixPath('selecao_xml_br/aa/v27n1/1809-4392-aa-27-1-0043.rep.xml'),
              PosixPath('selecao_xml_br/aa/v27n1/1809-4392-aa-27-1-0055.rep.xml'),
              PosixPath('selecao_xml_br/alea/v17n1/1517-106X-alea-17-01-00001.rep.xml'),
              PosixPath('selecao_xml_br/alea/v17n1/1517-106X-alea-17-01-00013.rep.xml'),
              PosixPath('selecao_xml_br/alea/v17n1/1517-106X-alea-17-01-00035.rep.xml'),
              PosixPath('selecao_xml_br/alea/v17n1/1517-106X-alea-17-01-00052.rep.xml'),
              PosixPath('selecao_xml_br/alea/v17n1/1517-106X-alea-17-01-00064.rep.xml'),
              PosixPath('se

              PosixPath('selecao_xml_br/iao/v18n1/1809-9777-iao-18-01-00043.rep.xml'),
              PosixPath('selecao_xml_br/iao/v18n1/1809-9777-iao-18-01-00049.rep.xml'),
              PosixPath('selecao_xml_br/iao/v18n1/1809-9777-iao-18-01-00054.rep.xml'),
              PosixPath('selecao_xml_br/iao/v18n1/1809-9777-iao-18-01-00057.rep.xml'),
              PosixPath('selecao_xml_br/iao/v18n1/1809-9777-iao-18-01-00063.rep.xml'),
              PosixPath('selecao_xml_br/iao/v18n1/1809-9777-iao-18-01-00068.rep.xml'),
              PosixPath('selecao_xml_br/iao/v18n1/1809-9777-iao-18-01-00077.rep.xml'),
              PosixPath('selecao_xml_br/iao/v18n1/1809-9777-iao-18-01-00080.rep.xml'),
              PosixPath('selecao_xml_br/iao/v18n1/1809-9777-iao-18-01-00083.rep.xml'),
              PosixPath('selecao_xml_br/iao/v18n1/1809-9777-iao-18-01-00087.rep.xml'),
              PosixPath('selecao_xml_br/iao/v19n1/1809-9777-iao-19-01-00001.rep.xml'),
              PosixPath('selecao_xml_br/iao

              PosixPath('selecao_xml_br/abcd/v28n1/0102-6720-abcd-28-01-00070.xml'),
              PosixPath('selecao_xml_br/abcd/v28n1/0102-6720-abcd-28-01-00074.xml'),
              PosixPath('selecao_xml_br/abcd/v28n1/0102-6720-abcd-28-01-00081.xml'),
              PosixPath('selecao_xml_br/abcd/v28n1/0102-6720-abcd-28-01-00086.xml'),
              PosixPath('selecao_xml_br/abcd/v28n1/0102-6720-abcd-28-01-00087.xml'),
              PosixPath('selecao_xml_br/abcd/v28n1/0102-6720-abcd-28-01-00089.xml'),
              PosixPath('selecao_xml_br/abcd/v28n1/0102-6720-abcd-28-01-00090.xml'),
              PosixPath('selecao_xml_br/abcd/v28s1/0102-6720-abcd-28-s1-00001.xml'),
              PosixPath('selecao_xml_br/abcd/v28s1/0102-6720-abcd-28-s1-00002.xml'),
              PosixPath('selecao_xml_br/abcd/v28s1/0102-6720-abcd-28-s1-00003.xml'),
              PosixPath('selecao_xml_br/abcd/v28s1/0102-6720-abcd-28-s1-00007.xml'),
              PosixPath('selecao_xml_br/abcd/v28s1/0102-6720-abcd

              PosixPath('selecao_xml_br/anp/v75n11/0004-282X-anp-75-11-0785.xml'),
              PosixPath('selecao_xml_br/anp/v75n11/0004-282X-anp-75-11-0789.xml'),
              PosixPath('selecao_xml_br/anp/v75n11/0004-282X-anp-75-11-0796.xml'),
              PosixPath('selecao_xml_br/anp/v75n11/0004-282X-anp-75-11-0801.xml'),
              PosixPath('selecao_xml_br/anp/v75n11/0004-282X-anp-75-11-0809.xml'),
              PosixPath('selecao_xml_br/anp/v75n11/0004-282X-anp-75-11-0813.xml'),
              PosixPath('selecao_xml_br/anp/v75n11/0004-282X-anp-75-11-0819.xml'),
              PosixPath('selecao_xml_br/anp/v75n11/0004-282X-anp-75-11-0825.xml'),
              PosixPath('selecao_xml_br/anp/v75n11/0004-282X-anp-75-11-0827.xml'),
              PosixPath('selecao_xml_br/anp/v75n11/0004-282X-anp-75-11-0830.xml'),
              PosixPath('selecao_xml_br/anp/v75n11/0004-282X-anp-75-11-0833.xml'),
              PosixPath('selecao_xml_br/anp/v75n11/0004-282X-anp-75-11-0835.xml'),
    

              PosixPath('selecao_xml_br/bjmbr/v46n01/bjb-46-01-058.original.xml'),
              PosixPath('selecao_xml_br/bjmbr/v46n01/bjb-46-01-058.xml'),
              PosixPath('selecao_xml_br/bjmbr/v46n01/bjb-46-01-071.original.xml'),
              PosixPath('selecao_xml_br/bjmbr/v46n01/bjb-46-01-071.xml'),
              PosixPath('selecao_xml_br/bjmbr/v46n01/bjb-46-01-079.original.xml'),
              PosixPath('selecao_xml_br/bjmbr/v46n01/bjb-46-01-079.xml'),
              PosixPath('selecao_xml_br/bjmbr/v46n01/bjb-46-01-085.original.xml'),
              PosixPath('selecao_xml_br/bjmbr/v46n01/bjb-46-01-085.xml'),
              PosixPath('selecao_xml_br/bjmbr/v46n01/bjb-46-01-091.original.xml'),
              PosixPath('selecao_xml_br/bjmbr/v46n01/bjb-46-01-091.xml'),
              PosixPath('selecao_xml_br/bjmbr/v46n01/bjb-46-01-098.original.xml'),
              PosixPath('selecao_xml_br/bjmbr/v46n01/bjb-46-01-098.xml'),
              PosixPath('selecao_xml_br/bjmbr/v46n1/1414-4

...

              PosixPath('selecao_xml_br/rbgg/v12n1/1981-2256-rbgg-12-01-00049.xml'),
              PosixPath('selecao_xml_br/rbgg/v12n1/1981-2256-rbgg-12-01-00063.xml'),
              PosixPath('selecao_xml_br/rbgg/v12n1/1981-2256-rbgg-12-01-00077.xml'),
              PosixPath('selecao_xml_br/rbgg/v12n1/1981-2256-rbgg-12-01-00087.xml'),
              PosixPath('selecao_xml_br/rbgg/v12n1/1981-2256-rbgg-12-01-00103.xml'),
              PosixPath('selecao_xml_br/rbgg/v12n1/1981-2256-rbgg-12-01-00113.xml'),
              PosixPath('selecao_xml_br/rbgg/v12n1/1981-2256-rbgg-12-01-00123.xml'),
              PosixPath('selecao_xml_br/rbgg/v17n1/1809-9823-rbgg-17-01-00007.xml'),
              PosixPath('selecao_xml_br/rbgg/v17n1/1809-9823-rbgg-17-01-00017.xml'),
              PosixPath('selecao_xml_br/rbgg/v17n1/1809-9823-rbgg-17-01-00027.xml'),
              PosixPath('selecao_xml_br/rbgg/v17n1/1809-9823-rbgg-17-01-00039.xml'),
              PosixPath('selecao_xml_br/rbgg/v17n1/1809-9823-rbgg

              PosixPath('selecao_xml_br/rca/v47n1/0045-6888-rca-47-01-0022.xml'),
              PosixPath('selecao_xml_br/rca/v47n1/0045-6888-rca-47-01-0032.xml'),
              PosixPath('selecao_xml_br/rca/v47n1/0045-6888-rca-47-01-0041.xml'),
              PosixPath('selecao_xml_br/rca/v47n1/0045-6888-rca-47-01-0047.xml'),
              PosixPath('selecao_xml_br/rca/v47n1/0045-6888-rca-47-01-0056.xml'),
              PosixPath('selecao_xml_br/rca/v47n1/0045-6888-rca-47-01-0069.xml'),
              PosixPath('selecao_xml_br/rca/v47n1/0045-6888-rca-47-01-0077.xml'),
              PosixPath('selecao_xml_br/rca/v47n1/0045-6888-rca-47-01-0086.xml'),
              PosixPath('selecao_xml_br/rca/v47n1/0045-6888-rca-47-01-0093.xml'),
              PosixPath('selecao_xml_br/rca/v47n1/0045-6888-rca-47-01-0101.xml'),
              PosixPath('selecao_xml_br/rca/v47n1/0045-6888-rca-47-01-0108.xml'),
              PosixPath('selecao_xml_br/rca/v47n1/0045-6888-rca-47-01-0118.xml'),
              Po

              PosixPath('selecao_xml_br/remi/v70n1/2448-167X-remi-70-01-0109.xml'),
              PosixPath('selecao_xml_br/remi/v71n1/2448-167X-remi-71-01-0005.xml'),
              PosixPath('selecao_xml_br/remi/v71n1/2448-167X-remi-71-01-0009.xml'),
              PosixPath('selecao_xml_br/remi/v71n1/2448-167X-remi-71-01-0019.xml'),
              PosixPath('selecao_xml_br/remi/v71n1/2448-167X-remi-71-01-0027.xml'),
              PosixPath('selecao_xml_br/remi/v71n1/2448-167X-remi-71-01-0037.xml'),
              PosixPath('selecao_xml_br/remi/v71n1/2448-167X-remi-71-01-0045.xml'),
              PosixPath('selecao_xml_br/remi/v71n1/2448-167X-remi-71-01-0053.xml'),
              PosixPath('selecao_xml_br/remi/v71n1/2448-167X-remi-71-01-0059.xml'),
              PosixPath('selecao_xml_br/remi/v71n1/2448-167X-remi-71-01-0067.xml'),
              PosixPath('selecao_xml_br/remi/v71n1/2448-167X-remi-71-01-0075.xml'),
              PosixPath('selecao_xml_br/remi/v71n1/2448-167X-remi-71-01-0081

              PosixPath('selecao_xml_br/rgenf/xml/tce/v23n1/0104-0707-tce-23-01-00211.xml'),
              PosixPath('selecao_xml_br/rgenf/xml/tce/v24n1/0104-0707-tce-24-01-00013.xml'),
              PosixPath('selecao_xml_br/rgenf/xml/tce/v24n1/0104-0707-tce-24-01-00022.xml'),
              PosixPath('selecao_xml_br/rgenf/xml/tce/v24n1/0104-0707-tce-24-01-00030.xml'),
              PosixPath('selecao_xml_br/rgenf/xml/tce/v24n1/0104-0707-tce-24-01-00038.xml'),
              PosixPath('selecao_xml_br/rgenf/xml/tce/v24n1/0104-0707-tce-24-01-00047.xml'),
              PosixPath('selecao_xml_br/rgenf/xml/tce/v24n1/0104-0707-tce-24-01-00055.xml'),
              PosixPath('selecao_xml_br/rgenf/xml/tce/v24n1/0104-0707-tce-24-01-00064.xml'),
              PosixPath('selecao_xml_br/rgenf/xml/tce/v24n1/0104-0707-tce-24-01-00072.xml'),
              PosixPath('selecao_xml_br/rgenf/xml/tce/v24n1/0104-0707-tce-24-01-00080.xml'),
              PosixPath('selecao_xml_br/rgenf/xml/tce/v24n1/0104-0707-

              PosixPath('selecao_xml_br/sausoc/v23n1/0104-1290-sausoc-23-01-00265.xml'),
              PosixPath('selecao_xml_br/sausoc/v23n1/0104-1290-sausoc-23-01-00277.xml'),
              PosixPath('selecao_xml_br/sausoc/v23n1/0104-1290-sausoc-23-01-00293.xml'),
              PosixPath('selecao_xml_br/sausoc/v23n1/0104-1290-sausoc-23-01-00302.xml'),
              PosixPath('selecao_xml_br/sausoc/v23n1/0104-1290-sausoc-23-01-00313.xml'),
              PosixPath('selecao_xml_br/sausoc/v23n1/0104-1290-sausoc-23-01-00325.xml'),
              PosixPath('selecao_xml_br/sausoc/v23n1/0104-1290-sausoc-23-01-00336.xml'),
              PosixPath('selecao_xml_br/sausoc/v24n1/0104-1290-sausoc-24-1-0009.xml'),
              PosixPath('selecao_xml_br/sausoc/v24n1/0104-1290-sausoc-24-1-0023.xml'),
              PosixPath('selecao_xml_br/sausoc/v24n1/0104-1290-sausoc-24-1-0034.xml'),
              PosixPath('selecao_xml_br/sausoc/v24n1/0104-1290-sausoc-24-1-0046.xml'),
              PosixPath('sele

Files that have an xref to an affiliation in a `<p>` element:
```
bjpp/v48n01/0102-3616-rbort-48-01-0057.xml
ld/v17n1/1518-7632-ld-17-01-00011.xml
pci/v21n1/1413-9936-pci-21-01-00134.xml
rbcdh/v17n1/1980-0037-rbcdh-17-01-00073.xml
ress/v25n1/2237-9622-ress-25-01-00095.xml
```

Files that have an xref to an affiliation in a `<name>` element:
```
aabc/v86n1/0001-3765-aabc-86-01-429.xml
```

That's the reason why Hugo Tourinho doesn't have an affiliation link in http://www.scielo.br/scielo.php?script=sci_arttext&pid=S0001-37652014000100429&lng=en&nrm=iso&tlng=en

Files that have an xref to an affiliation in an `<aff>` element:
```
ean/v18n1/1414-8145-ean-18-01-0075.xml
rgenf/xml/ean/v18n1/1414-8145-ean-18-01-0075.xml
```

There are duplicated files! These last two are actually the same!

In [31]:
!cmp -l selecao_xml_br/ean/v18n1/1414-8145-ean-18-01-0075.xml selecao_xml_br/rgenf/xml/ean/v18n1/1414-8145-ean-18-01-0075.xml